# Binning & Transition Dynamics

In [ ]:
from time import time
import numpy as np
from scipy import linalg as la
from scipy import stats as st
from scipy import optimize as opt
from matplotlib import pyplot as plt

from IPython.display import set_matplotlib_formats
%matplotlib inline
set_matplotlib_formats('svg')
plt.rcParams['figure.figsize'] = [8, 5]

## Binning (a.k.a., Non-Stochastic Simulations)

In [ ]:
class Huggett:


    def __init__(self, a_num, y_num, a_min=-3.0, a_max=15.0, beta=0.97,
                  gamma=2.0, mu=0.0, rho=0.53, sigma=0.296, vfi_tol=1e-6):
        self.na = a_num
        self.ny = y_num
        self.ns = a_num * y_num
        self.a_min = a_min
        self.beta = beta
        self.gamma = gamma
        self.mu = mu
        self.rho = rho
        self.sigma = sigma
        self.A = np.linspace(a_min, a_max, a_num)
        log_Y, self.Pi = self._rouwenhorst(y_num, mu, rho, sigma)
        self.Y = np.exp(log_Y)
        self.vfi_tol = vfi_tol


    def solve_vfi_ip(self, r):
        """
        Solves the households' problem with VFI scaling down the state space.
        """
        na = self.na // 5
        A = np.linspace(self.A.min(), self.A.max(), na)
        V0 = np.zeros((na, self.ny))
        dr = np.zeros((na, self.ny), dtype=int)
        crit = 1.0
        n_iter = 0
        while crit > self.vfi_tol:
            n_iter += 1
            V1 = np.zeros_like(V0)
            U = np.zeros((na, self.ny))
            for i in range(na):
                for k in range(self.ny):
                    C = self.Y[k] + (1 + r) * A[i] - A
                    C[C < 0] = np.nan
                    U[:, k] = self.u(C)
                objective = U + self.beta * ( V0 @ self.Pi.T )
                V1[i, :] = np.nanmax(objective, axis=0)
                dr[i, :] = np.nanargmax(objective, axis=0)
            crit = np.nanmax( np.nanmax( np.abs( V1 - V0 ) ) )
            V0[:] = V1
        pf_a = A[dr]
        A_opt = np.zeros((self.na, self.ny))
        for k in range(self.ny):
            coeffs = np.polyfit(A, pf_a[:, k], 3)
            A_opt[:, k] = np.polyval(coeffs, self.A)
        A_opt[A_opt <= self.A.min()] = self.A.min()
        return A_opt


    def market_clearing(self, r, binning=False, full_output=False):
        t0 = time()
        pfa = self.solve_vfi_ip(r)
        Q = self._compute_Q_smooth(pfa) if binning else self._compute_Q(pfa)
        dist = self._ergodic_distribution(Q).reshape((self.ny, self.na)).T
        net_excess_demand = np.sum(dist * pfa)
        t1 = time()
        print('Done!     r = {0:.5f}%     {1:.3f}s.'.format(r*100, t1-t0))
        if full_output:
            return net_excess_demand, dist
        else:
            return net_excess_demand


    def u(self, c):
        return (c ** (1 - self.gamma)) / (1 - self.gamma)


    @staticmethod
    def _rouwenhorst(n, mu, rho, sigma):
        """
        Discretizes any stationary AR(1) process.
        """
        def compute_P(p, n):
            if n == 2:
                P = np.array([[p, 1-p], [1-p, p]], dtype=float)
            else:
                Q = compute_P(p, n-1)
                A = np.zeros((n, n))
                B = np.zeros((n, n))
                A[:n-1, :n-1] += Q
                A[1:n, 1:n] += Q
                B[:n-1, 1:n] += Q
                B[1:n, :n-1] += Q
                P = p * A + (1-p) * B
                P[1:-1, :] /= 2
            return P
        p = (1 + rho) / 2
        Pi = compute_P(p, n)
        f = np.sqrt(n-1) * (sigma / np.sqrt(1 - rho**2))
        S = np.linspace(-f, f, n) + mu
        return S, Pi


    @staticmethod
    def _ergodic_distribution(P, tol=1e-12):
        """
        Returns the ergodic distribution of a matrix P by iterating it.
        (fast, if P is sparse)
        """
        n = P.shape[0]
        p0 = np.zeros((1, n))
        p0[0, 0] = 1.0
        diff = 1.0
        while diff > tol:
            p1 = p0 @ P
            p0 = p1 @ P
            diff = la.norm(p1 - p0)
        return p0.reshape((-1, )) / p0.sum()


    def _compute_Q(self, pf_a):
        """
        Translates a policy function into a transition matrix.
        """
        n = self.na
        blocks = []
        for k in range(self.ny):
            pa = np.zeros((n, n), dtype=int)
            for i in range(n):
                j = np.argmin( np.abs( pf_a[i, k] - self.A ) )
                pa[i, j] = 1
            blocks.append(pa)
        PA = la.block_diag(*blocks)
        PY = np.kron( self.Pi, np.eye(self.na) )
        Q = PY @ PA
        return Q


    def _compute_Q_smooth(self, pf_a):
        """
        Translates a policy function into a transition matrix, with binning.
        """
        pass

In [ ]:
mdl = Huggett(a_num=500, y_num=5)
rStar, checks = opt.ridder(mdl.market_clearing, 0.020, 0.025, full_output=True)

In [ ]:
checks

In [ ]:
pfa = mdl.solve_vfi_ip(rStar)
ned_bin,   dist_bin   = mdl.market_clearing(rStar, binning=True,  full_output=True)
ned_nobin, dist_nobin = mdl.market_clearing(rStar, binning=False, full_output=True)

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, sharex=True)
ax[0, 0].plot(mdl.A, dist_nobin.sum(axis=1))
ax[1, 0].plot(mdl.A, dist_nobin.sum(axis=1).cumsum())
ax[0, 1].plot(mdl.A, dist_bin.sum(axis=1))
ax[1, 1].plot(mdl.A, dist_bin.sum(axis=1).cumsum())
for j in range(2):
    ax[0, j].set_title('Ergodic marginal PDF')
    ax[1, j].set_title('Ergodic marginal CDF')
    for i in range(2):
        ax[i, j].grid(alpha=0.3)
plt.tight_layout()
plt.show()

_En passant,_ we have just replicated the paper by Huggett 😉

In [ ]:
print("  Complete-insurance economy: r = {:.3f}%.".format("???"))
print("Incomplete-insurance economy: r = {:.3f}%.".format(rStar * 100))

## Transition Dynamics (a.k.a., MIT shocks)

_Coming soon..._

## The Aiyagari (1994) Model

See the code provided by Maffezzoli.